In [1]:
import tempfile
import sys
import subprocess
import shutil
from os.path import join
from elasticsearch import Elasticsearch, helpers
from nir.utils import create_filter_query_function, change_bm25_parameters
from utils import *
import pickle
from mmnrm.dataset import TrainCollectionV2, TestCollectionV2
from mmnrm.evaluation import BioASQ_Evaluator

import json

es = Elasticsearch(["http://193.136.175.98:8125"])

index_name = "bioasq_9b"


In [2]:
queries = load_queries("training9b_wDates.json", maps=[("body","query")])

test_8b = []

for i in range(1,6):
    id_sets = set()
    with open(f"yearly_data/8B{i}_golden.json","r") as f:
        for q in json.load(f)["questions"]:
            id_sets.add(q["id"])
            
    test_8b.append(id_sets)

print(sum([len(x) for x in test_8b]))

queries_ids_sets = { x["id"] for x in queries }
train_ids = queries_ids_sets - test_8b[0]
validations_ids = test_8b[0]

train_data = subset_byId(queries, train_ids)
validation_data = subset_byId(queries, validations_ids)

convert_to_trainable_gs = lambda x: { k:{1:v}for k,v in x.items()}

train_data_queries, train_data_gs = separate_queries_goldstandard(train_data, additional_keys=["limit_date"])
train_data_gs = convert_to_trainable_gs(train_data_gs)
print(len(train_data))
validation_data_queries, validation_data_gs = separate_queries_goldstandard(validation_data, additional_keys=["limit_date"])
print(len(validation_data))

500
3643
100


In [3]:
# make a BM25 search

experiments = [(0.9, 0.4, 100), (0.6, 0.4, 250)]

for K1,BETA,TOP_N in experiments:

    query_results = execute_search(es, train_data_queries, TOP_N, index_name, k1=K1, b=BETA)

    t_collection = TrainCollectionV2(train_data_queries, 
                                   train_data_gs, 
                                   query_results, 
                                   use_relevance_groups=False)\
                            .batch_size(32)

    t_collection.save("training_batch_01_"+str(TOP_N))

    ## VALIDATION DATA

    query_results = execute_search(es, validation_data_queries, TOP_N, index_name, k1=K1, b=BETA)

    validation_data_gs = list(map(lambda x:{"id":x["id"], 
                                            "query":x["query"], 
                                            "documents":[y.split("/")[-1] for y in x["documents"]],
                                            "limit_date":x["limit_date"]},
                                           validation_data))

    evaluator = BioASQ_Evaluator(validation_data_gs)

    validation_collection = TestCollectionV2(validation_data_queries, 
                                               query_results,
                                               evaluator)\
                                        .batch_size(32)

    validation_collection.save("validation_data_batch_01_"+str(TOP_N))

Setting the k1 and b for BM25
Minimum number of relevance type(0) in the queries of the goldstandard sub set: 43
Mean number of relevance type(0) in the queries of the goldstandard sub set: 93.25704225352112
Minimum number of relevance type(1) in the queries of the goldstandard sub set: 1
Mean number of relevance type(1) in the queries of the goldstandard sub set: 6.742957746478873
Sub Collection size 278351
Number of skipped question, due to lack of true positives 235
Setting the k1 and b for BM25
Setting the k1 and b for BM25
Minimum number of relevance type(0) in the queries of the goldstandard sub set: 163
Mean number of relevance type(0) in the queries of the goldstandard sub set: 242.2012910798122
Minimum number of relevance type(1) in the queries of the goldstandard sub set: 1
Mean number of relevance type(1) in the queries of the goldstandard sub set: 7.798708920187793
Sub Collection size 649700
Number of skipped question, due to lack of true positives 0
Setting the k1 and b fo